In [3]:
import pandas as pd

# Load the CSV file
df_pool = pd.read_csv('Pool.csv', low_memory=False)
df_rib = pd.read_csv('RIB.csv', low_memory=False)

In [4]:
display(df_pool)
print(df_pool.dtypes)
display(df_rib)
print(df_rib.dtypes)

,PCR SL,Ext. Batch SL,Pool Set,Sample_ID,Batch No,RSV,FluA,FluB,Result
0,1,1,P-1,12403015541,245 Zm,NaN,NaN,NaN,Negative
1,2,2,P-1,12403015651,245 Zm,NaN,25.9,NaN,Positive
2,3,3,P-1,12403014652,245 Zm,NaN,NaN,10.3,Positive
3,4,4,P-1,12403013713,245 Zm,19.4,NaN,NaN,Positive


PCR SL             int64
Ext. Batch SL      int64
Pool Set          object
Sample_ID          int64
Batch No          object
RSV              float64
FluA             float64
FluB             float64
Result            object
dtype: object


,Sl no,Collection Date,Study ID,Name,Phone no.,Relation with contact person,Lab_ID,Unnamed: 7,Collecting nurse,Results,Test date,Informed results by call,Informed results by text,Name of RA,Remarks,Enroll in COI (Y/N),Reason for non enrollment,Linking done (Y/N)
0,1,19/11/22,12403015651,elham,NaN,NaN,NaN,Sick,NaN,FluA(25.9),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
1,2,19/11/22,12403014652,fariya,1.730947e+09,father,1.221102e+10,NaN,mahbuba,FluB(10.3),21/11/2022,NaN,Ok,Shahina,NaN,Y,NaN,Y
2,3,19/11/22,12403013713,saifullah,1.301757e+09,father,1.221102e+10,NaN,mahbuba,RSV (19.4),21/11/2022,NaN,Ok,Shahina,NaN,Y,NaN,Y
3,4,19/11/22,1070004622,anas,1.711134e+09,father,1.221103e+10,NaN,mahbuba,RSV (19.70),21/11/2022,Ok,Ok,Shahina,NaN,NaN,NaN,Y
4,5,19/11/22,1070004623,avary,1.304730e+09,father,1.221103e+10,NaN,mahbuba,Negative,21/11/2022,NaN,Ok,Shahina,NaN,NaN,NaN,Y
5,6,19/11/22,1070004613,morsalin,NaN,NaN,NaN,Discharge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
6,7,19/11/22,12403015541,surjo,1.745063e+09,father,1.221103e+10,NaN,mahbuba,Negative,23/11/2022,NaN,NaN,NaN,NaN,Y,NaN,Y


Sl no                             int64
Collection Date                  object
Study ID                          int64
Name                             object
Phone no.                       float64
Relation with contact person     object
Lab_ID                          float64
Unnamed: 7                       object
Results                          object
Test date                        object
Informed results by call         object
Informed results by text         object
Name of RA                       object
Remarks                         float64
Enroll in COI (Y/N)              object
Reason for non enrollment       float64
Linking done (Y/N)               object
dtype: object


In [11]:
import pandas as pd

# Load the datasets
pool_df = pd.read_csv('Pool.csv')
rib_df = pd.read_csv('RIB.csv')

# Correct data types by converting 'Lab_ID' in RIB.csv to integer (if necessary, depending on the data)
rib_df['Lab_ID'] = rib_df['Lab_ID'].astype('Int64')

# Drop rows where 'Lab_ID' is NaN in RIB.csv
rib_cleaned = rib_df.dropna(subset=['Lab_ID'])

# Ensure correct column names, stripping any extra whitespace
rib_cleaned.columns = rib_cleaned.columns.str.strip()

# Map the 'Results' from RIB.csv to Pool.csv based on matching 'Sample_ID' (Pool.csv) and 'Lab_ID' (RIB.csv)
pool_df['PCR_Results'] = pool_df['Sample_ID'].map(rib_cleaned.set_index('Lab_ID')['Results'])

# Save the updated Pool.csv DataFrame if needed
pool_df.to_csv('Updated_Pool.csv', index=False)

# Display the first few rows of the updated DataFrame to verify
print(pool_df.head())


   PCR SL  Ext. Batch SL Pool Set    Sample_ID Batch No PCR_Results
0       1              1      P-1  12403015541   245 Zm  FluA(25.9)
1       2              2      P-1  12403015651   245 Zm  FluB(10.3)
2       3              3      P-1  12403014652   245 Zm  RSV (19.4)
3       4              4      P-1  12403013713   245 Zm         NaN


In [13]:
import pandas as pd

# Load the data
pool_df = pd.read_csv('Pool.csv', dtype={'Sample_ID': 'int64'})
rib_df = pd.read_csv('RIB.csv', dtype={'Lab_ID': 'Int64', 'Results': 'object'})

# Clean up the column names to ensure there's no trailing whitespace
rib_df.columns = rib_df.columns.str.strip()

# Drop rows where 'Lab_ID' is NaN to ensure clean merging
rib_cleaned = rib_df.dropna(subset=['Lab_ID'])

# Convert IDs to integer for precise matching
pool_df['Sample_ID'] = pool_df['Sample_ID'].astype('int64')
rib_cleaned['Lab_ID'] = rib_cleaned['Lab_ID'].astype('int64')

# Merge data based on 'Sample_ID' from Pool.csv and 'Lab_ID' from RIB.csv
# Specify 'left' to keep all rows from Pool.csv and pull in matching rows from RIB.csv
merged_df = pd.merge(pool_df, rib_cleaned, left_on='Sample_ID', right_on='Lab_ID', how='left')

# Select relevant columns to avoid unnecessary memory usage, especially if rib_df is large
final_df = merged_df[['PCR SL', 'Ext. Batch SL', 'Pool Set', 'Sample_ID', 'Batch No', 'Results']]

# Rename the 'Results' column to 'PCR_Results'
final_df.rename(columns={'Results': 'PCR_Results'}, inplace=True)

# Save the merged DataFrame
final_df.to_csv('Updated_Pool.csv', index=False)

# Display the first few rows to verify
print(final_df.head())

   PCR SL  Ext. Batch SL Pool Set    Sample_ID Batch No PCR_Results
0       1              1      P-1  12403015541   245 Zm  FluA(25.9)
1       2              2      P-1  12403015651   245 Zm  FluB(10.3)
2       3              3      P-1  12403014652   245 Zm  RSV (19.4)
3       4              4      P-1  12403013713   245 Zm         NaN


/tmp/ipykernel_43215/862450303.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rib_cleaned['Lab_ID'] = rib_cleaned['Lab_ID'].astype('int64')
/tmp/ipykernel_43215/862450303.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.rename(columns={'Results': 'PCR_Results'}, inplace=True)
